# 1. Data import and pre processing.

Create the Spark Environment and read in the data set.

In [42]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import Row
from pyspark.sql.types import *       # for datatype conversion
from pyspark.sql.functions import *   # for col() function
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import Tokenizer
import re
import pyspark.sql.functions as f
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from numpy import array
from math import sqrt
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml.classification import LogisticRegression
from pyspark.sql.types import DoubleType
from pyspark.ml.feature import StringIndexer

sc = SparkContext.getOrCreate()
sqlCtx = SQLContext(sc)

spark = SparkSession \
    .builder \
    .master("local") \
    .appName("Jeopardy Calculation") \
    .config("spark.executor.memory", '2g') \
    .config('spark.executor.cores', '1') \
    .config('spark.cores.max', '1') \
    .config("spark.driver.memory",'1g') \
    .getOrCreate()

jeopardy = spark.read.json('data/JEOPARDY_QUESTIONS1.json')

In [18]:
jeopardy.show(20)

+----------+--------------------+--------------------+--------------------+---------+-----------+-----+
|  air_date|              answer|            category|            question|    round|show_number|value|
+----------+--------------------+--------------------+--------------------+---------+-----------+-----+
|2004-12-31|          Copernicus|             HISTORY|'For the last 8 y...|Jeopardy!|       4680| $200|
|2004-12-31|          Jim Thorpe|ESPN's TOP 10 ALL...|'No. 2: 1912 Olym...|Jeopardy!|       4680| $200|
|2004-12-31|             Arizona|EVERYBODY TALKS A...|'The city of Yuma...|Jeopardy!|       4680| $200|
|2004-12-31|         McDonald\'s|    THE COMPANY LINE|'In 1963, live on...|Jeopardy!|       4680| $200|
|2004-12-31|          John Adams| EPITAPHS & TRIBUTES|'Signer of the De...|Jeopardy!|       4680| $200|
|2004-12-31|             the ant|      3-LETTER WORDS|'In the title of ...|Jeopardy!|       4680| $200|
|2004-12-31|      the Appian Way|             HISTORY|'Built in 

First we wanted to observe how many observations were in each category.

In [14]:
jeopardy.columns

['air_date', 'answer', 'category', 'question', 'round', 'show_number', 'value']

In [16]:
j_categoryCount = jeopardy.groupBy("category").count()
j_categoryCount.sort(desc("count")).show(5)

+----------------+-----+
|        category|count|
+----------------+-----+
|  BEFORE & AFTER|  547|
|         SCIENCE|  519|
|      LITERATURE|  496|
|AMERICAN HISTORY|  418|
|       POTPOURRI|  401|
+----------------+-----+
only showing top 5 rows



In [20]:
j_categoryCount.count()

27995

The most populated categories are each approximately 0.21% of the original data set, which is extremely sparse, especially given that there are roughly 28,000 distinct categories. 
Therefore, instead of constructing models for the entire data set, we will filter out categories with less than 100 observations.

In [28]:
count100 = j_categoryCount.sort(desc("count")).filter(j_categoryCount["count"] > 100).count()
count100

145

This shows that there are 145 categories with more than 100 observations. 

In [29]:
top_categories = list(j_categoryCount.sort(desc("count")).select("category").limit(count100).toPandas().category)

In [30]:
jeo_f = jeopardy.where(col("category").isin(top_categories))
jeo_f.show(5)

+----------+--------------------+--------------+--------------------+---------+-----------+-----+
|  air_date|              answer|      category|            question|    round|show_number|value|
+----------+--------------------+--------------+--------------------+---------+-----------+-----+
|2004-12-31|          Copernicus|       HISTORY|'For the last 8 y...|Jeopardy!|       4680| $200|
|2004-12-31|             the ant|3-LETTER WORDS|'In the title of ...|Jeopardy!|       4680| $200|
|2004-12-31|      the Appian Way|       HISTORY|'Built in 312 B.C...|Jeopardy!|       4680| $400|
|2004-12-31|             the cud|3-LETTER WORDS|'Cows regurgitate...|Jeopardy!|       4680| $400|
|2004-12-31|Ceylon (or Sri La...|       HISTORY|'In 1000 Rajaraja...|Jeopardy!|       4680| $600|
+----------+--------------------+--------------+--------------------+---------+-----------+-----+
only showing top 5 rows



In [31]:
jeo_f.count()

29252

Now that we have narrowed down our data set, we are able to process the data by stripping the punctuation and tokenizing the questions into individual words.

In [32]:
#stripping punctuation, tokenizing, and stop word removing for the modified dataset
jeo_fpunc = jeo_f.withColumn("stripped", f.regexp_replace(f.col("question"), "[\!@#$%^&*)(><,';:]", ""))
jeo_fpunc.cache()
tokenizer = Tokenizer(inputCol = "stripped", outputCol = "words")
tokenized_f = tokenizer.transform(jeo_fpunc)
tokenized_f.cache()
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
jeo_fStopRemoved = remover.transform(tokenized_f)
jeo_fStopRemoved.cache()

DataFrame[air_date: string, answer: string, category: string, question: string, round: string, show_number: string, value: string, stripped: string, words: array<string>, filtered: array<string>]

In [34]:
from nltk.stem.porter import *
stemmer = PorterStemmer()

def stem(in_vec):
    out_vec = []
    for t in in_vec:
        t_stem = stemmer.stem(t)
        out_vec.append(t_stem)       
    return out_vec

from pyspark.sql.types import *
stemmer_udf = udf(lambda x: stem(x), ArrayType(StringType()))
#stemming words in modified dataset
jeo_fStemmed = jeo_fStopRemoved.withColumn("stemmed", stemmer_udf("filtered"))
jeo_fStemmed.cache()

DataFrame[air_date: string, answer: string, category: string, question: string, round: string, show_number: string, value: string, stripped: string, words: array<string>, filtered: array<string>, stemmed: array<string>]

The data has now been stemmed and has the following attributes:

In [37]:
jeo_fStemmed.columns

['air_date',
 'answer',
 'category',
 'question',
 'round',
 'show_number',
 'value',
 'stripped',
 'words',
 'filtered',
 'stemmed']

In [38]:
jeo_fStemmed.show(5, truncate=False)

+----------+---------------------+--------------+--------------------------------------------------------------------------------------------------+---------+-----------+-----+-----------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------+-----------------------------------------------------------------------+
|air_date  |answer               |category      |question                                                                                          |round    |show_number|value|stripped                                                                                       |words                                                                                                              |filtered                                                                  |s

In [35]:
hashingTF = HashingTF(inputCol="stemmed", outputCol="rawFeatures", numFeatures=50000)
featurizedData = hashingTF.transform(jeo_fStemmed)
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)
rescaledData.cache()

DataFrame[air_date: string, answer: string, category: string, question: string, round: string, show_number: string, value: string, stripped: string, words: array<string>, filtered: array<string>, stemmed: array<string>, rawFeatures: vector, features: vector]

In [ ]:
rescaledData.show(2)

In [39]:
indexer = StringIndexer(inputCol="category", outputCol="label")
indexed = indexer.fit(rescaledData).transform(rescaledData)
indexed.select("label","category").show(truncate=False)

+-----+-----------------+
|label|category         |
+-----+-----------------+
|8.0  |HISTORY          |
|42.0 |3-LETTER WORDS   |
|8.0  |HISTORY          |
|42.0 |3-LETTER WORDS   |
|8.0  |HISTORY          |
|42.0 |3-LETTER WORDS   |
|8.0  |HISTORY          |
|42.0 |3-LETTER WORDS   |
|8.0  |HISTORY          |
|42.0 |3-LETTER WORDS   |
|81.0 |IN THE DICTIONARY|
|81.0 |IN THE DICTIONARY|
|81.0 |IN THE DICTIONARY|
|81.0 |IN THE DICTIONARY|
|81.0 |IN THE DICTIONARY|
|77.0 |TRAVEL & TOURISM |
|77.0 |TRAVEL & TOURISM |
|77.0 |TRAVEL & TOURISM |
|77.0 |TRAVEL & TOURISM |
|77.0 |TRAVEL & TOURISM |
+-----+-----------------+
only showing top 20 rows



Split the data into training and test sets.

In [40]:
training,test = indexed.randomSplit([0.8,0.2], seed = 1) 

In [ ]:
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lrModel = lr.fit(training)